In [94]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim 

In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/emnist-balanced-train.csv')
df_test=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/emnist-balanced-test.csv')


image_data = df_train.iloc[2, 1:].values.reshape(28, 28)
plt.imshow(image_data, cmap='gray')
plt.show()

In [96]:
x_train=df_train.drop("45",axis=1)
y_train=df_train.iloc[:,0].values

x_test=df_test.drop("41",axis=1)
y_test=df_test.iloc[:,0].values

data_train=torch.tensor(x_train.values)
lable_train=torch.tensor(y_train)


data_test=torch.tensor(x_test.values)
lable_test=torch.tensor(y_test)

In [97]:
dataset_train=TensorDataset(data_train, lable_train)
dataset_test=TensorDataset(data_test, lable_test)

In [98]:
data_train_loader= DataLoader(dataset_train, batch_size=5, shuffle=True)
data_test_loader= DataLoader(dataset_test, batch_size=5, shuffle=True)


In [123]:

#3 layer neuralnetwork

class Net(nn.Module):
    def __init__(self):
        super( Net, self).__init__()
        self.hidden_layer_1 = nn.Linear(784, 256)
        self.hidden_layer_2 = nn.Linear(256, 32)
        self.output_layer = nn.Linear(32, 47)
        self.activation = nn.Sigmoid()

    def forward(self, x):
        x = x.to(torch.float32)
        x = self.activation(self.hidden_layer_1(x))
        x = self.activation(self.hidden_layer_2(x))
        x = self.output_layer(x)
        return x

In [120]:

#5 layer neural network

class Net(nn.Module):
    def __init__(self):
        super( Net, self).__init__()
        self.hidden_layer_1 = nn.Linear(784, 256)
        self.hidden_layer_2 = nn.Linear(256, 128)
        self.hidden_layer_3 = nn.Linear(128, 64)
        self.hidden_layer_4 = nn.Linear(64, 32)
        self.output_layer = nn.Linear(32, 47)
        self.activation = nn.Sigmoid()

    def forward(self, x):
        x = x.to(torch.float32)
        x = self.activation(self.hidden_layer_1(x))
        x = self.activation(self.hidden_layer_2(x))
        x = self.activation(self.hidden_layer_3(x))
        x = self.activation(self.hidden_layer_4(x))
        x = self.output_layer(x)
        return x

In [127]:
# 3 layer neural network with drop out


dropout_rate = 0.4
class Net(nn.Module):
    def __init__(self):
        super( Net, self).__init__()
        self.hidden_layer_1 = nn.Linear(784, 256)
        self.dropout1 = nn.Dropout(dropout_rate)
        self.hidden_layer_2 = nn.Linear(256, 32)
        self.dropout2 = nn.Dropout(dropout_rate)
        self.output_layer = nn.Linear(32, 47)
        self.activation = nn.Sigmoid()

    def forward(self, x):
        x = x.to(torch.float32)
        x = self.activation(self.hidden_layer_1(x))
        x = self.dropout1(x)
        x = self.activation(self.hidden_layer_2(x))
        x = self.dropout2(x)
        x = self.output_layer(x)
        return x

In [133]:
 #3 layer neural network with batch normalization



class Net(nn.Module):
    def __init__(self):
        super( Net, self).__init__()
        self.hidden_layer_1 = nn.Linear(784, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.hidden_layer_2 = nn.Linear(256, 32)
        self.bn2 = nn.BatchNorm1d(32)
        self.output_layer = nn.Linear(32, 47)
        self.activation = nn.Sigmoid()

    def forward(self, x):
        x = x.to(torch.float32)
        x=self.hidden_layer_1(x)
        x = self.bn1(x)
        x = self.activation(x)
        x=self.hidden_layer_2(x)
        x = self.bn2(x)
        x = self.activation(x)
        x = self.output_layer(x)
        return x

In [134]:
net=Net()

criterion=nn.CrossEntropyLoss()
optimizer=optim.SGD(net.parameters(),lr=0.001,momentum=0.9)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
net.to(device)

In [ ]:

no_epochs = 10
for epoch in range(no_epochs):
    for i, data in enumerate(data_train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        print(f'epoch:{epoch + 1}, batch: {i+1},  loss: {loss.item()}')

In [ ]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in data_test_loader:
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        # calculate outputs by running images through the network
        outputs = net(inputs)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 5 test images: {100 * correct / total} %')